In [2]:
import astropy.coordinates as coord
import astropy.table as at
import astropy.units as u
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np

# gala
import gala.coordinates as gc
import gala.dynamics as gd
import gala.potential as gp
from gala.units import galactic

- Sky coordinates from 
- Proper motions from Table 2: (Blue sample) https://arxiv.org/pdf/2012.09204.pdf
- Distance and RV from Mcconachie 2012

In [3]:
m31_c = coord.SkyCoord(
    ra=10.68470833 * u.deg, 
    dec=41.26875 * u.deg,
    distance=731 * u.kpc,
    pm_ra_cosdec=48.98 * u.microarcsecond/u.yr, 
    pm_dec=-36.85 * u.microarcsecond/u.yr,
    radial_velocity=-300 * u.km/u.s
)

In [8]:
class HalocentricLocalGroup(coord.BaseCoordinateFrame):
    """
    Position at the Milky Way Halo barycenter, x-axis toward M31, 
    z-axis toward the Local Group angular momentum vector.
    """

    default_representation = coord.CartesianRepresentation
    default_differential = coord.CartesianDifferential

    # Frame attributes
    m31_coord = coord.CoordinateAttribute(
        frame=coord.ICRS,
        default=m31_c
    )
    
    M_MW_over_M_M31 = coord.Attribute(
        default=0.5
    )
    
    M_LG = coord.QuantityAttribute(
        default=2e12*u.Msun,
        unit=u.Msun
    )

In [94]:
def get_galcen_to_lg_transform(lg_frame, galcen_frame, inverse=False):
    """
    This function returns the matrix and (position and velocity) offset 
    vectors to transform from a Milky Way Galactocentric reference frame to
    a Local Group Barycenter reference frame.
    
    Astropy coordinate frames are mostly defined as affine transformations from 
    one frame to another. Transformations between inertial frames are given by 
    affine transformations, which are defined as A x + b, where A is a matrix 
    (typically a rotation matrix: orthogonal matrix with determinant 1), 
    x and b are vectors.
    
    Use the ``inverse`` argument to get the inverse transformation, matrix and
    offsets to go from LGBarycentric to Galactocentric.
    
    """
    # shorthand
    lg = lg_frame
    M_M31 = lg.M_LG / (1 + lg.M_MW_over_M_M31)
    M_MW = lg.M_LG - M_M31
    
    # Get the line connecting M31 to MW center, and angular momentum 
    # vector to specify the orientation / rotation around the line
    m31_galcen = lg.m31_coord.transform_to(galcen_frame)
    m31_galcen_pos = m31_galcen.data.without_differentials()
    m31_galcen_vel = m31_galcen.velocity    
    L_mw_m31 = m31_galcen_pos.cross(m31_galcen_vel)
    lg_pole = L_mw_m31 / L_mw_m31.norm()

    # Rotation matrix to align x(Galcen) with the vector to M31 and 
    # z(Galcen) with the LG angular momentum vector
    new_x = m31_galcen_pos / m31_galcen_pos.norm()
    new_z = lg_pole
    new_y = - new_x.cross(new_z)
    R = coord.concatenate_representations((new_x, new_y, new_z)).xyz.T
    
    # Positional offset to the barycenter
    # - This is defined already in the LG frame!
    dpos = coord.CartesianRepresentation(lg.m31_coord.distance * [1., 0, 0])
    
    # Velocity offset to the barycenter
    # - This is defined in the MW frame!
    dvel = - m31_galcen_vel * M_M31 / lg.M_LG
    
    if inverse:
        inv_dpos = (-dpos).transform(R.T)
        b = inv_dpos.with_differentials(-dvel)
        A = R.T
        
    else:
        b = dpos.with_differentials(dvel.transform(R))
        A = R
    
    return A, b


@coord.frame_transform_graph.transform(
    coord.transformations.AffineTransform, 
    coord.Galactocentric, 
    LGBarycentric
)
def galactocentric_to_lg(galactocentric_coord, lg_frame):
    return get_galcen_to_lg_transform(lg_frame, galactocentric_coord, inverse=False)


@coord.frame_transform_graph.transform(
    coord.transformations.AffineTransform, 
    LGBarycentric,
    coord.Galactocentric
)
def lg_to_galactocentric(lg_coord, galactocentric_frame):
    return get_galcen_to_lg_transform(lg_coord, galactocentric_frame, inverse=True)

In [95]:
lg_frame = LGBarycentric()
galcen_frame = coord.Galactocentric()

In [96]:
mw_c = coord.SkyCoord(
    coord.CartesianRepresentation([0,0,0]*u.kpc).with_differentials(coord.CartesianDifferential([0,0,0]*u.km/u.s)),
    frame=galcen_frame
)

In [97]:
m31_c.transform_to(LGBarycentric())

<SkyCoord (LGBarycentric: m31_coord=<ICRS Coordinate: (ra, dec, distance) in (deg, deg, kpc)
    (10.68470833, 41.26875, 731.)
 (pm_ra_cosdec, pm_dec, radial_velocity) in (mas / yr, mas / yr, km / s)
    (0.04898, -0.03685, -300.)>, M_MW_over_M_M31=0.5, M_LG=2000000000000.0 solMass): (x, y, z) in kpc
    (1465.94425169, 5.68434189e-14, -1.42108547e-14)
 (v_x, v_y, v_z) in km / s
    (-37.8361322, 19.51181308, -4.50111254e-15)>

In [99]:
mw_c.transform_to(LGBarycentric())

<SkyCoord (Galactocentric: galcen_coord=<ICRS Coordinate: (ra, dec) in deg
    (266.4051, -28.936175)>, galcen_distance=8.122 kpc, galcen_v_sun=(12.9, 245.6, 7.78) km / s, z_sun=20.8 pc, roll=0.0 deg): (x, y, z) in kpc
    (0., 0., 0.)
 (v_x, v_y, v_z) in km / s
    (-3.55271368e-15, 0., 2.66453526e-15)>

In [100]:
m31_c.transform_to(LGBarycentric()).transform_to(galcen_frame)

<SkyCoord (Galactocentric: galcen_coord=<ICRS Coordinate: (ra, dec) in deg
    (266.4051, -28.936175)>, galcen_distance=8.122 kpc, galcen_v_sun=(12.9, 245.6, 7.78) km / s, z_sun=20.8 pc, roll=0.0 deg): (x, y, z) in kpc
    (-360.6982349, 581.62813522, -267.85919575)
 (v_x, v_y, v_z) in km / s
    (41.58355717, -120.5840702, -6.39089485)>